In [7]:
import pandas as pd

def recommend_movies(input_movie_title, movies_df):
    
    # Normalizing input title for case-insensitive comparison
    normalized_input_title = input_movie_title.lower().strip()

    base_movie_row = movies_df[movies_df['title'].str.lower().str.contains(normalized_input_title, regex=False)]

    # If multiple movies match, try to find an exact match or pick the first one
    if len(base_movie_row) > 1:
        exact_match = base_movie_row[base_movie_row['title'].str.lower() == normalized_input_title]
        if not exact_match.empty:
            base_movie_row = exact_match
        else:
            # If no exact match among multiples, picking the first one
            base_movie_row = base_movie_row.iloc[[0]]

    if base_movie_row.empty:
        print(f"Sorry, \"{input_movie_title}\" was not found in our database. Please try another movie.")
        return []
    
        # Get the genres of the base movie
    base_movie_genres = base_movie_row['genres'].iloc[0]
    base_movie_id = base_movie_row['MovieID'].iloc[0] # Using MovieID to exclude the movie itself

    # Filtering out the base movie itself from the recommendations
    other_movies_df = movies_df[movies_df['MovieID'] != base_movie_id].copy()

    # Calculating similarity based on shared genres
    other_movies_df['score'] = other_movies_df['genres'].apply(
        lambda movie_genres: sum(1 for genre in base_movie_genres if genre in movie_genres)
    )

    # Sorting movies by score in descending order (most shared genres first)
    # Taking only top 6 recommendations
    recommended_df = other_movies_df.sort_values(by='score', ascending=False).head(6)

    # Converting the recommended DataFrame rows back to a list of dictionaries
    # for consistent output format with previous versions.
    recommendations = recommended_df[['title', 'genres']].to_dict(orient='records')

    return recommendations

#Main Execution Block
if __name__ == "__main__":
    #Data Loading and Preprocessing
    try:
        # Loading movies.dat
        movies_df = pd.read_csv(
            'movies.dat',
            sep='::',
            engine='python',
            header=None, 
            names=['MovieID', 'title', 'genres'], 
            encoding='latin-1'
        )
        print("Dataset 'movies.dat' loaded successfully.")
        
        print(movies_df.head())

    except FileNotFoundError:
        print("Error: 'movies.dat' not found. Please download the MovieLens 1M dataset")
        print("from https://grouplens.org/datasets/movielens/1m/ and place 'movies.dat'")
        print("in the same directory as this script, or provide the correct path.")
        exit() # Exit if the file isn't found
    except Exception as e:
        print(f"An error occurred during file loading: {e}")
        exit()

   

 # Data Cleaning and Preprocessing
    # 1. Droping rows with missing values in 'title' or 'genres'
    movies_df.dropna(subset=['title', 'genres'], inplace=True)

    # 2. Filtering out movies with '(no genres listed)'
    movies_df = movies_df[movies_df['genres'] != '(no genres listed)']

    # 3. Converting genre strings to lists of genres
    movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

    print(f"Dataset preprocessed. Total movies available: {len(movies_df)}")
    print("-" * 50)

    while True:
        user_input = input("Enter a movie title (or 'quit' to exit): ")
        if user_input.lower() == 'quit':
            break

        # Passing the preprocessed DataFrame to the recommendation function
        recommendations = recommend_movies(user_input, movies_df)

        if recommendations:
            print(f"\nRecommendations for \"{user_input}\":")
            for i, movie in enumerate(recommendations):
                print(f"{i+1}. {movie['title']} (Genres: {', '.join(movie['genres'])})")
            print("-" * 50)
        else:
            print("\nNo recommendations could be made. Please try a different movie or check your spelling.")
            print("-" * 50)

Dataset 'movies.dat' loaded successfully.
   MovieID                               title                        genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
Dataset preprocessed. Total movies available: 3883
--------------------------------------------------
Enter a movie title (or 'quit' to exit): Jumanji (1995)

Recommendations for "Jumanji (1995)":
1. Indian in the Cupboard, The (1995) (Genres: Adventure, Children's, Fantasy)
2. Star Kid (1997) (Genres: Adventure, Children's, Fantasy, Sci-Fi)
3. Pagemaster, The (1994) (Genres: Action, Adventure, Animation, Children's, Fantasy)
4. Space Jam (1996) (Genres: Adventure, Animation, Children's, Com